In [1]:
'''
Author: OCEAN.GZY
Date: 2022-09-18 15:49:17
LastEditors: OCEAN.GZY
LastEditTime: 2022-09-18 15:50:53
FilePath: \ML\nlp\聊天机器人\tf2.6_prj\test.ipynb
Description: 注释信息
'''
import json
import os
import sys
import time
import tensorflow as tf
import io

In [2]:
tf.config.run_functions_eagerly(True)

In [3]:
from configparser import SafeConfigParser

In [4]:
conf_file = os.getcwd() + "/seq2seq.ini"
if not os.path.exists(conf_file):
    conf_file = os.path.dirname(os.getcwd() + "/seq2seq.ini")


def get_config():
    print(conf_file)
    parser = SafeConfigParser()
    parser.read(conf_file, encoding='utf-8')
    _conf_ints = [(key, int(value)) for key, value in parser.items('ints')]
    _conf_floats = [(key, float(value)) for key, value in parser.items('floats')]
    _conf_strings = [(key, str(value)) for key, value in parser.items('strings')]
    return dict(_conf_ints + _conf_floats + _conf_strings)

In [5]:
# 初始化超参字典
gConf = {}
gConf = get_config()

# 通过超参字典为vocab_in_size,vocab_tar_size,embedding_dim,units等赋值
vocab_inp_size = gConf["vocab_inp_size"]
vocab_tar_size = gConf["vocab_tar_size"]
embedding_dim = gConf["embedding_dim"]
units = gConf["layer_size"]
BATCH_SIZE = gConf["batch_size"]

max_length_inp = gConf["max_length"]
max_length_tar = gConf["max_length"]
log_dir = gConf["log_dir"]

e:\GitHub\ML\nlp\聊天机器人\tf2.6_prj/seq2seq.ini


c:\Users\Administrator\Envs\py3.6.5_tf2\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.
  


In [6]:
# 定义encoder类
class Encoder(tf.keras.Model):
    # 初始化参数，对默认参数进行初始化

    def __init__(self, vocab_size, emdedding_dim, encode_units, batch_size):
        """
        @param vocab_size:
        @param emdedding_dim:
        @param encode_units:
        @param batch_size:

        :param vocab_size: 非重复的词汇总数
        :param embedding_dim: 词嵌入的维度
        :enc_units: 编码器中GRU层的隐含节点数
        :batch_sz: 数据批次大小(每次参数更新用到的数据量)
        """
        super(Encoder, self).__init__()
        self.encode_units = encode_units
        self.batch_size = batch_size
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        """
               return_sequences：
                       布尔值。是返回输出序列中的最后一个输出还是完整序列。 默认值：False。
                       True代表返回GRU序列模型的每个时间步的输出(每个输出做连接操作)
               return_state：
                       布尔值。 除输出外，是否返回最后一个状态。 默认值：False。
                       True代表除了返回输出外，还需要返回最后一个隐层状态。
               recurrent_initializer：
                       recurrent_kernel权重矩阵的初始化程序，用于对递归状态进行线性转换。 默认值：正交。
                       'glorot_uniform'即循环状态张量的初始化方式为均匀分布。
        """
        # 实例化gru层
        # return_sequences=True代表返回GRU序列模型的每个时间步的输出(每个输出做连接操作)
        # return_state=True代表除了返回输出外，还需要返回最后一个隐层状态
        # recurrent_initializer='glorot_uniform'即循环状态张量的初始化方式为均匀分布
        self.gru = tf.keras.layers.GRU(self.encode_units, return_sequences=True, return_state=True,
                                       recurrent_initializer="glorot_uniform")

    # 定义调用函数
    def call(self, x, hidden):
        """
        @param x:
        @param hidden:
        @return:
        """
        # 对输入进行embedding操作
        x_embedding = self.embedding(x)
        """initial_state：要传递给单元格的第一个调用的初始状态张量的列表（可选，默认为None，这将导致创建零填充的初始状态张量）。"""
        # 通过gru层获得最后一个时间步的输出和隐含状态
        output, state = self.gru(x_embedding, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self):
        """ (BATCH_SIZE, 隐藏层中的隐藏神经元数量) """
        # gru层的隐含节点对应的参数张量以零张量初始化
        return tf.zeros((self.batch_size, self.encode_units))


# 定义bahdanauAttention类，bahdanauAttention是常用的attention实现方法之一
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        # 注意力网络的初始化
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    """
    传入值：
        features：编码器的输出，(64, 16, 1024) 即 (BATCH_SIZE, 输入序列最大长度句子的长度, 隐藏层中的隐藏神经元数量)
        hidden：解码器的隐层输出状态，(64, 1024) 即 (batch_size, hidden_size) (BATCH_SIZE, 隐藏层中的隐藏神经元数量)
    返回值：
        attention_result：(64, 1024) 即 (batch size, units) (BATCH_SIZE, 隐藏层中的隐藏神经元数量)
        attention_weights：(64, 16, 1) 即 (batch_size, sequence_length, 1) (BATCH_SIZE, 输入序列最大长度句子的长度, 1)
    """

    def call(self, features, hidden):
        """
        description: 具体计算函数
        :param features: 编码器的输出
        :param hidden: 解码器的隐层输出状态
        return: 通过注意力机制处理后的结果和注意力权重attention_weights
        """
        """
        1.hidden_with_time_axis = tf.expand_dims(hidden, 1)
                解码器的隐层输出状态hidden，(64, 1024) 即 (batch_size, hidden_size) (BATCH_SIZE, 隐藏层中的隐藏神经元数量)。
                hidden扩展一个维度从(64, 1024)变成(64, 1,1024)。
        2.score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))
                计算注意力得分score。
                features：编码器的输出，(64, 16, 1024)。
                hidden_with_time_axis：解码器的隐层输出状态，(64, 1,1024)
                W1和W2：Dense(隐藏层中的隐藏神经元数量1024)
                tanh(W1(features) + W2(hidden_with_time_axis))：
                ---> tanh(W1((64, 16, 1024)) + W2((64, 1,1024)))
                ---> tanh((64, 16, 1024))
                ---> (64, 16, 1024) 即 (BATCH_SIZE, 输入序列最大长度句子的长度, 隐藏层中的隐藏神经元数量)
        3.attention_weights = tf.nn.softmax(self.V(score), axis=1)
                计算注意力权重attention_weights。
                V：Dense(隐藏层中的隐藏神经元数量1)
                softmax(V(score), axis=1)
                ---> softmax(V((64, 16, 1024)), axis=1)
                ---> softmax((64, 16, 1), axis=1)
                ---> (64, 16, 1) 即 (BATCH_SIZE, 输入序列最大长度句子的长度, 1)
                因为注意力得分score的形状是(BATCH_SIZE, 输入序列最大长度句子的长度, 隐藏层中的隐藏神经元数量)，
                输入序列最大长度句子的长度(max_length)是输入的长度。
                因为我们想为每个输入长度分配一个权重，所以softmax应该用在第一个轴(max_length)上axis=1，
                而softmax默认被应用于最后一个轴axis=-1。
        4.context_vector = tf.reduce_sum(attention_weights * features, axis=1)
                获得注意力机制处理后的结果context_vector。
                reduce_sum(attention_weights * features, axis=1)
                ---> reduce_sum((64, 16, 1) * (64, 16, 1024), axis=1)
                ---> reduce_sum((64, 16, 1024), axis=1)
                ---> (64, 1024) 即 (BATCH_SIZE, 隐藏层中的隐藏神经元数量)
        """

        # 将hidden增加一个维度,(batch_size, hidden_size) --> (batch_size, 1, hidden_size)
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        # 根据公式计算注意力得分, 输出score的形状为: (batch_size, 16, hidden_size)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))
        # 根据公式计算注意力权重, 输出attention_weights形状为: (batch_size, 16, 1)
        attention_weights = tf.nn.softmax(self.V(score), axis=1)
        # 最后根据公式获得注意力机制处理后的结果context_vector
        # context_vector的形状为: (batch_size, hidden_size)
        context_vector = attention_weights * features
        # 将乘机后的context_vector按行相加，进行压缩得到最终的context_vector
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights


"""
构建RNN解码器：这里RNN是指GRU, 同时在解码器中使用注意力机制.
"""


class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, decode_units, batch_size):
        super(Decoder, self).__init__()
        # 初始化batch_size、decode_units、embedding 、gru 、fc、attention
        self.batch_size = batch_size
        self.decode_units = decode_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.decode_units, return_sequences=True, return_state=True,
                                       recurrent_initializer="glorot_uniform")

        # 实例化一个Dense层作为输出层
        self.fc = tf.keras.layers.Dense(vocab_size)
        # 在解码器阶段我们将使用注意力机制，这里实例化注意力的类
        self.attention = BahdanauAttention(self.decode_units)

    """
    1.x = self.embedding(x)
            输入：(64, 1) 64行1列，批量大小句子数为64，1列为该行句子的第N列的单词
            输出：(64, 1, 256) (BATCH_SIZE, 输入序列最大长度句子的长度, 嵌入维度)
    2.context_vector, attention_weights = self.attention(hidden, enc_output)
            attention_weights注意力权重：(64, 16, 1) 即 (BATCH_SIZE, 输入序列最大长度句子的长度, 1)
            context_vector注意力机制处理后的结果：(64, 1024) 即 (BATCH_SIZE, 隐藏层中的隐藏神经元数量)
    3.x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
            tf.expand_dims(context_vector, 1)：(64, 1, 1024) 即 (BATCH_SIZE, 1, 隐藏层中的隐藏神经元数量)
            concat([(64, 1, 1024),(64, 1, 256)], axis=-1)：1024+256=1280，最终输出 (64, 1, 1280)
    4.GRU
        1.tf.keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
            return_sequences：
                    布尔值。是返回输出序列中的最后一个输出还是完整序列。 默认值：False。
                    True代表返回GRU序列模型的每个时间步的输出(每个输出做连接操作)
            return_state：
                    布尔值。 除输出外，是否返回最后一个状态。 默认值：False。
                    True代表除了返回输出外，还需要返回最后一个隐层状态。
            recurrent_initializer：
                    recurrent_kernel权重矩阵的初始化程序，用于对递归状态进行线性转换。 默认值：正交。
                    'glorot_uniform'即循环状态张量的初始化方式为均匀分布。
        2.output, state = gru(x)      
            output：
                    (64, 1, 1024) 即 (BATCH_SIZE, 1, 隐藏层中的隐藏神经元数量)
                    (当前批次的样本个数, 当前样本的序列长度(单词个数), 隐藏层中神经元数量 * 1)
            state：
                    (64, 1024) 即 (BATCH_SIZE, 隐藏层中的隐藏神经元数量)
    5.output = tf.reshape(output, (-1, output.shape[2]))
             (-1, output.shape[2])：表示把(64, 1, 1024)转换为(64, 1024) 即 (BATCH_SIZE, 隐藏层中的隐藏神经元数量)
    6.x = self.fc(output)
            x：(64, 4935) 即 (BATCH_SIZE, 目标序列的不重复单词的总数作为目标序列的字典大小)
    """

    def call(self, x, hidden, encode_ouput):
        # print("x.shape",x.shape) #(64, 1)。64行1列，批量大小句子数为64，1列为该行句子的第N列的单词

        # 对decoder的输入通过embedding层
        x = self.embedding(x)
        # print("x1.shape",x.shape) #(64, 1, 256)。(BATCH_SIZE, 输入序列最大长度句子的长度, 嵌入维度)

        # 使用注意力规则计算hidden与enc_output的'相互影响程度(计算attention，输出上下文语境向量)
        context_vector, attention_weights = self.attention(encode_ouput, hidden)
        # print("tf.expand_dims(context_vector, 1).shape",tf.expand_dims(context_vector, 1).shape) #(64, 1, 1024)

        # 将这种'影响程度'与输入x拼接(这个操作也是注意力计算规则的一部分)（拼接上下文语境与decoder的输入embedding，并送入gru中）
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        # print("x2.shape",x.shape) #(64, 1, 1280)

        # 将新的x输入到gru层中得到输出
        output, state = self.gru(x)
        # print("output1.shape",output.shape) #(64, 1, 1024) 即 (BATCH_SIZE, 1, 隐藏层中的隐藏神经元数量)
        # print("state.shape",state.shape) #(64, 1024) 即 (BATCH_SIZE, 隐藏层中的隐藏神经元数量)

        # 改变输出形状使其适应全连接层的输入形式
        output = tf.reshape(output, (-1, output.shape[2]))
        # print("output2.shape",output.shape) #(64, 1024) 即 (BATCH_SIZE, 隐藏层中的隐藏神经元数量)

        # 使用全连接层作为输出层
        # 输出的形状 == （批大小，vocab）
        x = self.fc(output)
        # print("x3.shape",x.shape) #(64, 4935) 即 (BATCH_SIZE, 目标序列的不重复单词的总数作为目标序列的字典大小)

        return x, state, attention_weights

    def initialize_hidden_state(self):
        return tf.zeros(self.batch_size, self.decode_units)


In [7]:
# 对训练语料进行处理，上下文分别加上start end表示
def preprocess_sentence(w):
    w = 'start ' + w + ' end'
    return w

In [8]:
# 定义word2number函数，通过对语料的处理提取词典，并进行word2number处理以及padding补全
def tokenize(vocab_file):
    # 从词典中读取预先生成tokenizer的config，构建词典矩阵
    with open(vocab_file, 'r', encoding="utf-8") as f:
        tokenize_config = json.dumps(json.load(f), ensure_ascii=False)
        lang_tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(tokenize_config)

    # 利用词典进行word2number的转换以及padding处理
    return lang_tokenizer

In [9]:
# 实例化encoder、decoder、optimizer、checkpoint等
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
optimizer = tf.keras.optimizers.Adam()
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

In [91]:
# 定义预测函数，根据上下文预测下文对话
def predict(sentence):
    # 从词典中读取预先 生成的tokenizer的config， 构建词典矩阵
    input_tokenizer = tokenize(gConf["vocab_inp_path"])
    target_tokenizer = tokenize(gConf["vocab_tar_path"])

    # 加载预训练模型
    checkpoint_dir = gConf["model_data"]
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
    # 对输入字句进行处理，加上 start end标示
    # sentence = preprocess_sentence(sentence)

    # 进行word2number的转换
    inputs = input_tokenizer.texts_to_sequences(tuple([sentence]))
    # 进行padding补全
    inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''

    # 初始化一个中间状态
    hidden = [tf.zeros((1, units))]
    # 对输入上文进行encoder编码，提取特征
    encoder_out, encoder_hidden = encoder(inputs, hidden)
    decoder_hidden = encoder_hidden
    # decoder的输入从 start的对应id 开始正向输入
    decoder_input = tf.expand_dims([target_tokenizer.word_index['start']], 0)
    # 在最大的语句长度范围内，使用模型中的decoder进行循环解码
    for t in range(max_length_tar):
        # 获得解码结果，并使用argmax确定概率最大的id
        predictions, decoder_hidden, attention_weights = decoder(decoder_input, decoder_hidden,encoder_out)
        predicted_id = tf.argmax(predictions[0].numpy())
        # 判断当前id是否为 语句结束表示， 如果是则停止循环解码， 否则进行number2word的转换，并进行语句的拼接
        if target_tokenizer.index_word[predicted_id.numpy()] == 'end':
            break

        result += str(target_tokenizer.index_word[predicted_id.numpy()]) + ' '
        # 将预测得到的id作为下一个时刻的decoder输入
        decoder_input = tf.expand_dims([predicted_id.numpy()], 0)
    return result

In [12]:
input_tokenizer = tokenize(gConf["vocab_inp_path"])
input_tokenizer
target_tokenizer = tokenize(gConf["vocab_tar_path"])
target_tokenizer
# 加载预训练模型
checkpoint_dir = gConf["model_data"]
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
checkpoint

In [59]:
input_tokenizer.split

' '

In [60]:
lines = io.open("./datasets/seq.data", encoding="utf-8").read().strip().split("\n")
word_pairs = [[preprocess_sentence(w) for w in l.split("\t")] for l in lines]
input_lang, target_lang = zip(*word_pairs)

In [61]:
word_pairs

[['start 呵呵 end', 'start 是 王若 猫 的 end'],
 ['start 不是 end', 'start 那 是 什么 end'],
 ['start 怎么 了 end', 'start 我 很 难过 安慰 我 ~ end'],
 ['start 开心 点哈 , 一切 都 会 好 起来 end', 'start 嗯 end'],
 ['start 我 还 喜欢 她 , 怎么办 end', 'start 我 帮 你 告诉 她 发短信 还是 打电话 end'],
 ['start 短信 end', 'start 嗯 嗯 我 也 相信 end'],
 ['start 你 知道 谁 么 end', 'start 肯定 不是 我 是 阮德培 end'],
 ['start 许兵 是 谁 end', 'start 吴院 四班 小帅哥 end'],
 ['start 这么 假 end', 'start 三鹿 奶粉 也 假 不 一样 的 卖 啊 end'],
 ['start 许兵 是 傻 逼 end', 'start 被 你 发现 了 end'],
 ['start 许兵 是 谁 end', 'start 是 我 善良 可爱 的 主人 的 老公 啊 end'],
 ['start 许兵 是 谁 end', 'start 是 穆森 的 老婆 啊 end'],
 ['start 许兵 是 谁 end', 'start 奇葩 end'],
 ['start 许兵 到底 是 谁 end', 'start 不要 凶 我 应该 大概 也许 是 叶祺 吧 end'],
 ['start 尼玛 , 许兵 到底 是 谁 end', 'start 我 我 不会 告诉 你 我 是 小澜 的 ( ^ V ^ ) end'],
 ['start 小黄 鸭 , 你 有 女朋友 么 end', 'start 老娘 是 女 的 end'],
 ['start 那 你 有 男朋友 么 end', 'start 没有 呢 我 只要 主人 一个 人 疼爱 我 嘛 end'],
 ['start 那 你 在 哪 end', 'start 我 无聊 end'],
 ['start 你 妈 是 谁 end', 'start 我 妈妈 是 世界 上 最 温柔 的 主人 你 啊 end'],
 ['s

In [62]:
input_lang

('start 呵呵 end',
 'start 不是 end',
 'start 怎么 了 end',
 'start 开心 点哈 , 一切 都 会 好 起来 end',
 'start 我 还 喜欢 她 , 怎么办 end',
 'start 短信 end',
 'start 你 知道 谁 么 end',
 'start 许兵 是 谁 end',
 'start 这么 假 end',
 'start 许兵 是 傻 逼 end',
 'start 许兵 是 谁 end',
 'start 许兵 是 谁 end',
 'start 许兵 是 谁 end',
 'start 许兵 到底 是 谁 end',
 'start 尼玛 , 许兵 到底 是 谁 end',
 'start 小黄 鸭 , 你 有 女朋友 么 end',
 'start 那 你 有 男朋友 么 end',
 'start 那 你 在 哪 end',
 'start 你 妈 是 谁 end',
 'start 去 你 大爷 的 end',
 'start 你 在 骂 我 一句 end',
 'start 你 大爷 的 end',
 'start 你 是 屌丝 鸡 end',
 'start 高富帅 end',
 'start 你 end',
 'start 呵呵 end',
 'start 今天 是 谁 的 生日 end',
 'start 你 敢不敢 end',
 'start 呵呵 end',
 'start 呵呵 呵 end',
 'start 你 是 女 的 了 ? 怎么回事 end',
 'start 呵呵 end',
 'start 天王 盖地 虎 end',
 'start 小通 end',
 'start 在 监考 , 你 在 干 么 end',
 'start 哼 ! 你 不想 我 我 就 不 和 你 玩 end',
 'start 你 要 气死我 吗 ? 坏蛋 end',
 'start 恩 , 也 是 ! 那 我 不能 生气 啦 ! 你 生气 就行 end',
 'start 你 谈过 恋爱 么 end',
 'start 什么 让 你 这么 伤心 end',
 'start 敢 问 你 的 性别 end',
 'start 小受 end',
 'start 是 吗 end',


In [65]:
input_lang[:2]

('start 呵呵 end', 'start 不是 end')

In [64]:
input_tokenizer.texts_to_sequences(input_lang[:2])

[[3, 37, 2], [3, 33, 2]]

In [72]:
type(input_lang[:2])

tuple

In [74]:
tuple(['start 你 是 谁 呀 end'])

('start 你 是 谁 呀 end',)

In [75]:
input_tokenizer.texts_to_sequences(tuple(['start 你 是 谁 呀 end']))

[[3, 4, 8, 14, 168, 2]]

In [81]:
inputs = input_tokenizer.texts_to_sequences(tuple(['start 你 是 谁 呀 end']))
print(inputs)
inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
print(inputs.shape)
print(inputs)
inputs = tf.convert_to_tensor(inputs)
print(inputs.shape)
print(inputs)

[[3, 4, 8, 14, 168, 2]]
(1, 20, 6)
[[[  3   4   8  14 168   2]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]]]
(1, 20, 6)
tf.Tensor(
[[[  3   4   8  14 168   2]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   0   0   0]
  [  0   0   0   

In [83]:
inputs = input_tokenizer.texts_to_sequences(tuple(['start 你 是 谁 呀 end']))
print(inputs)
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=max_length_inp, padding='post')
print(inputs.shape)
print(inputs)
inputs = tf.convert_to_tensor(inputs)
print(inputs.shape)
print(inputs)

[[3, 4, 8, 14, 168, 2]]
(1, 20)
[[  3   4   8  14 168   2   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]]
(1, 20)
tf.Tensor(
[[  3   4   8  14 168   2   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]], shape=(1, 20), dtype=int32)


In [94]:
# 初始化一个中间状态
hidden = [tf.zeros((1, units))]
print(hidden)
hidden

[<tf.Tensor: shape=(1, 256), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0

[<tf.Tensor: shape=(1, 256), dtype=float32, numpy=
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,


In [95]:
# 对输入上文进行encoder编码，提取特征
encoder_out, encoder_hidden = encoder(inputs, hidden)

In [96]:
encoder_out

<tf.Tensor: shape=(1, 20, 256), dtype=float32, numpy=
array([[[ 1.6069721e-01, -8.9798309e-04,  9.2845492e-04, ...,
         -3.8693422e-03, -5.2476805e-02,  1.9238135e-02],
        [ 1.7645290e-01,  3.7359868e-03, -4.4910272e-04, ...,
         -6.3664811e-03, -5.7450328e-02, -2.9659012e-02],
        [ 1.8365403e-01,  4.5272291e-02, -7.3425718e-02, ...,
         -7.9396078e-03, -6.0326569e-02, -4.4968981e-01],
        ...,
        [ 9.9999303e-01, -3.6435631e-01,  1.7032517e-01, ...,
          9.9980515e-01,  1.0000000e+00,  9.9999517e-01],
        [ 9.9998981e-01, -3.6457378e-01,  1.6568726e-01, ...,
          9.9985212e-01,  1.0000000e+00,  9.9999636e-01],
        [ 9.9998575e-01, -3.6432990e-01,  1.6258408e-01, ...,
          9.9988127e-01,  1.0000000e+00,  9.9999654e-01]]], dtype=float32)>

In [97]:
encoder_hidden

<tf.Tensor: shape=(1, 256), dtype=float32, numpy=
array([[ 9.99985754e-01, -3.64329904e-01,  1.62584081e-01,
         9.92738605e-01,  1.42102558e-02,  2.39417523e-01,
         5.84844708e-01,  4.11996841e-01, -9.99087751e-01,
        -4.62627364e-03, -5.34493141e-02,  8.27720523e-01,
         9.99999166e-01,  5.98106682e-01, -1.80987510e-04,
         9.99914289e-01, -9.99999881e-01, -9.99999583e-01,
         9.99923825e-01,  2.52684236e-01,  1.85230151e-01,
         9.99997020e-01,  4.95396219e-02, -8.99218917e-01,
        -9.99997616e-01,  1.43688023e-01,  6.22135162e-01,
         4.01705176e-01, -9.48665943e-03, -6.17418051e-01,
        -9.66542363e-01, -2.23764494e-01,  9.99999583e-01,
        -9.99931097e-01, -1.00000000e+00,  9.79224682e-01,
        -9.97642040e-01, -9.99998927e-01, -9.69037771e-01,
         1.90609917e-02, -9.99582946e-01,  9.67245936e-01,
        -5.84018640e-02,  3.05181183e-02, -4.98135269e-01,
         5.10247290e-01, -7.59902418e-01, -1.10667214e-01,
      

In [92]:
import jieba

In [93]:
predict(" ".join(jieba.cut("你好呀")))

'我 也 是 '

In [98]:
tf.saved_model

<module 'tensorflow._api.v2.saved_model' from 'c:\\Users\\Administrator\\Envs\\py3.6.5_tf2\\lib\\site-packages\\tensorflow\\_api\\v2\\saved_model\\__init__.py'>

In [100]:
saver = tf.compat.v1.train.Saver()

RuntimeError: When eager execution is enabled, `var_list` must specify a list or dict of variables to save

In [ ]:
s